# ANALYSIS

In [26]:
import pandas as pd
import ast
import numpy as np

# Load the csv file

In [27]:
df = pd.read_csv('best-books.csv') 

# Change the awards col into num of awards:

In [28]:
df2 = df.head(50)
df2 = df2.replace({'awards': {np.nan: 0}})

d = []

for i in df2.index:
    if df2.loc[i, "awards"] == 0:
        d.append(0)
    else:
        d.append( len( ast.literal_eval(df2.loc[i, "awards"]) ) )

df2['awards length'] = d
# df2['awards length'].sum()

# min max normalization

In [29]:
df_min_max = df2['avg_rating'].copy()
def min_max_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.min()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

minmax_norm_ratings = min_max_normalization(df_min_max)
# print(minmax_norm_ratings)
df2['minmax_norm_ratings'] = minmax_norm_ratings
# print(df2)


# mean normalization

In [30]:
def mean_normalization(df_min_max):
    df_min_max = 1 + (df_min_max - df_min_max.mean()) / (df_min_max.max() - df_min_max.min()) * 9
    return round(df_min_max, 2)

mean_norm_ratings = mean_normalization(df_min_max)
df2['mean_norm_ratings'] = mean_norm_ratings
# print(df2)

# sort by minmax_norm_ratings Given Author

In [31]:
def check_author(df, author):
    df = df.loc[df.author == author]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_author(df2, 'J.K. Rowling')

# sort by publishing year and get mean of minmax_norm_ratings

In [ ]:
df2.groupby('original_publish_year')['minmax_norm_ratings'].mean()

# sort by author and check who has highest average rating

In [ ]:
df2.groupby('author')['minmax_norm_ratings'].mean().sort_values( ascending=False)

# sort by genre and check ratings

In [ ]:
# NEED TO SEE HOW THIS BEHAVES WITH 3 VALUES=3 GENRES
df2.groupby('genres')['minmax_norm_ratings'].mean().sort_values( ascending=False)

# sort no of awards by author

In [ ]:
# df2['awards length']
df2.sort_values(by=['awards length', 'author'], ascending=[False, True])

# sort by minmax_norm_ratings Given Year of publication

In [ ]:
def check_year(df, year):
    df = df.loc[df.original_publish_year == year]
    df.sort_values(by=['minmax_norm_ratings'], ascending=[False])
    return df

# check_year(df2, 2005)

# check if book series are rated better than non series books

In [ ]:
df2.groupby('series')['minmax_norm_ratings'].mean()

# WORK IN PROGRESS - check what place is more commonly reffered to in these books

In [36]:
# df2 = df2.replace({'places': {'nan': 0}})
df2 = df2.replace({'places': {np.nan: 0}})
# print(df2['places'])

dictt = []
for i in df2.index:
    if df2.loc[i, "places"] == 0:
        dictt.append(0)
    else:
        dictt.append( ast.literal_eval(df2.loc[i, "places"]) )

df2['places_new'] = dictt
print(df2['places_new'])




# df2.groupby('places').size()

0           [District 12, Panem, Capitol, Panem, Panem]
1     [Hogwarts School of Witchcraft and Wizardry, L...
2                           [Maycomb, Alabama, Alabama]
3        [United Kingdom, Derbyshire, England, England]
4     [Forks, Washington, Phoenix, Arizona, Washingt...
5                                   [Molching, Germany]
6                             [England, United Kingdom]
7                                     [London, England]
8                                        [Middle-earth]
9                    [Indianapolis, Indiana, Amsterdam]
10                                   [Atlanta, Georgia]
11                                                    0
12                                                    0
13                                                    0
14                                            [England]
15                     [Paris, London, England, France]
16           [Thornfield Hall, England, United Kingdom]
17                                       [Kyoto,